In [7]:
#必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import datetime
import time
import cmath
%matplotlib inline
import warnings
warnings.simplefilter('ignore')
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.animation as anm

In [ ]:
#グラフからLaplacian行列を生成する関数
def create_l(G):
    global w,l
    w=np.array(nx.to_numpy_matrix(G))
    l=-w
    for i in range(0,len(w)):
        l[i][i]=w[i].sum()


In [1]:
dt=1/100
enter=5000




###----入力部1--------------------
path=500
#グラフの生成
G=nx.path_graph(path)
#パスグラフのリンクの重みを一括で指定
weight=1
#特異点を入力するノードの指定（ノードの両方向へ特異点を伝えるため2ノード必要）
design_node1=0

#吸収用ノードの番号指定
absorb_node1=len(G)
G.add_nodes_from([absorb_node1])
#吸収用ノードとソースノードの連結
G.add_edges_from([(design_node1,absorb_node1)])

fig = plt.figure(figsize = (10,10))
nx.draw_networkx(G)
#→グラフ図をみたい場合fig.show()
#グラフからLaplacian行列を生成
create_l(G)
###---------------------------

start = time.time()
###-----入力部２---------------

#実行時間
times=dist=500
#Δtの値（刻み幅）
dt=dt
#特異点の最大値（頂点）の値の指定

#経過時間測定の幅
hop=500

###-----------------------


##シミュレーションの下準備
top_edge=1
#逆数を整数値に変換
d_dt=int(1/dt)
d_weight=int(1/weight)
#先にデータフレイムのハコの用意
x=pd.DataFrame(index=np.arange(0,d_dt*times),columns=np.arange(0,len(l)))
#先に空きデータを0で埋めておく
x.fillna(0,inplace=True)
x=x.astype(float)
#総ステップ数を表示
print('Step Amount:{}'.format(d_dt*times))
enter=enter
#ソースノードに状態量を与える(特異点)

#ソースノードに状態量を与える(特異点)
for design_node in [design_node1]:
    for a in range(0,enter+1):
        x.loc[a,design_node]=a*top_edge*dt*dt
        x.loc[enter*2-a,design_node]=a*top_edge*dt*dt
"""
design_node=design_node1
x.loc[0,design_node]=0
x.loc[1,design_node]=1
x.loc[2,design_node]=0
"""


##シミュレーション開始
#時間設置(dtから始めることで，for tにおいて，t＝dtの時にt＋dtを計算している)
for t in range(1,times*d_dt):

    #ノードiについてのループ
    for i in np.arange(0,len(l)):
        A_sum=0

        #ノードiを固定のもと，ノードjを回すことで，隣接関係を計算
        for j in np.arange(0,len(l)):
            if w[i][j]!=0:
                A_sum=A_sum+w[i][j]*weight*(x[i][t]-x[j][t])

        #特異点入力中のソースノードと吸収用ノードの挙動（design_nodeとabsorb_node)    
        #本実験では，吸収用ノードは2つ
        if  i == absorb_node1  :


            #特異点入力中は吸収用ノードは0で固定
            if t<2*enter+4:

                x.loc[t+1,absorb_node1]=0


            #特異点入力後，吸収用ノードは観測ノードの挙動を追尾するように動かす
            else:

                x.loc[t+1,absorb_node1]=x[design_node1][t+1-int(d_dt/np.sqrt(weight))]

        #特異点入力中は，ソースノードは差分法に従わせずに動かす．          
        elif i == design_node1 :

            if t<2*enter+1:
                pass

            #特異点入力後は，ソースノードも差分法に従う
            else:
                x.loc[t+1,i]=-x[i][t-1]+2*x[i][t]-dt*dt*A_sum

        #ソースノードと吸収用ノード以外のノードは，全て差分法に従う
        else:
            x.loc[t+1,i]=-x[i][t-1]+2*x[i][t]-dt*dt*A_sum

    if t in np.arange(0,d_dt*times+hop,hop):
        process_time = time.time() - start
        if t==hop:
            print('predict_time={}m'.format(round((d_dt*times/t)*process_time/60)))
            clock=datetime.datetime.fromtimestamp(time.time()+(d_dt*times/t)*process_time)
            print("predict_clock={}:{}".format(clock.hour,clock.minute))

        print("step={}:process_time={}min".format(t,round(process_time/60)))
print('Real_time:{}min'.format(round((d_dt*times/t)*process_time/60)))
    #------------------------------------

#名前をつけてcsvファイルとして出力→このデータを基にVis.pyで可視化
x.to_csv('path-{}-{}-{}.csv'.format(path,d_dt,enter*2))
print('finish')

NameError: name 'nx' is not defined